In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torchsummary import summary
from torch.nn.functional import mse_loss
from torch.utils.data import Dataset, DataLoader
from sklearn.ae_selection import train_test_split
from utils_pp import replace_cell_names_with_id
%load_ext autoreload
%autoreload 2

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

ModuleNotFoundError: No module named 'sklearn.ae_selection'

In [ ]:
# columns = ["cell_line", "drugA_name", "drugB_name", "drugA_conc", "drugB_conc", "target"]
# df_train = pd.read_csv("../data_raw/oneil.csv", usecols=(1,2,3,4,5,12)).iloc[:,[0,1,3,2,4,5]].set_axis(columns, axis=1)
# df_train["cell_line"]
# df_test = pd.read_csv("../data/test_yosua.csv").set_axis(columns + ["std"], axis=1).convert_dtypes()

drug_data = pd.read_pickle("../data/drug_data.pkl.compress", compression="gzip")
cell_data = pd.read_pickle("../data/cell_line_data.pkl.compress", compression="gzip")
cell_data_val = cell_data.copy()
X_train_cell, X_val_cell, y_train_cell, y_val_cell = train_test_split(cell_data, cell_data_val, test_size=0.2)
# df_train = replace_cell_names_with_id(dataframe=df_train, mapping_file="../data/mappingccl.csv")
# df_test = replace_cell_names_with_id(dataframe=df_test, mapping_file="../data/mappingccl.csv")
# df_train = df_train[df_train.cell_line.isin(cell_data.index)]
# df_train, df_val = train_test_split(df_train, test_size=0.2, shuffle=True)
# cell_data = cell_data[cell_data.index.isin(pd.concat([df_train.cell_line, df_test.cell_line]))]
# print("oneil", df_train.memory_usage().sum()/1e6, df_train.shape,"\n", df_train.dtypes)
print("drug_feat", X_train_cell.memory_usage().sum()/1e6, X_train_cell.shape)
print("drug_feat", X_val_cell.memory_usage().sum()/1e6, X_val_cell.shape)


NameError: name 'pd' is not defined

In [43]:
class dataset(Dataset):
    def __init__(self, data, labels):
        self.dataset = data.to_numpy()
        self.labels = labels.to_numpy()

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx], self.labels[idx]
                
train_set_drug  = dataset(X_train_cell, y_train_cell)
val_set_drug  = dataset(X_val_cell, y_val_cell)
train_dl_drug = DataLoader(train_set_drug, batch_size=1, shuffle=True)
val_dl_drug = DataLoader(val_set_drug, batch_size=1, shuffle=True)
xi, yi = next(iter(train_dl_drug))
print(xi.shape, yi.shape)

torch.Size([1, 2416]) torch.Size([1, 2416])


In [45]:

# Creating a PyTorch class
# 28*28 ==> 9 ==> 28*28
class AE(torch.nn.Module):
    def __init__(self, h_sizes):
        super().__init__()
         
        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 784 ==> 9
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(h_sizes[0], h_sizes[1]),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(h_sizes[1], h_sizes[2])
        )
         
        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # The Sigmoid activation function
        # outputs the value between 0 and 1
        # 9 ==> 784
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(h_sizes[2], h_sizes[3]),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(h_sizes[3], h_sizes[0])
        )
 
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
ae = AE([xi.shape[1],512,256,512])
yi = ae.forward(xi)
print(xi.shape, yi.shape)

torch.Size([1, 2416]) torch.Size([1, 2416])


In [52]:
def train_one_epoch(epoch_index, tb_writer, training_loader, optimizer, loss_fn, verbose=False):
    running_loss = 0.
    last_loss = 0.
    for i, data in enumerate(training_loader):
        inputs, labels = data
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        outputs = ae(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        running_loss += loss.item() / inputs.shape[0]
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            if verbose:print('  batch {} loss: {}'.format(i + 1, last_loss), outputs[0][0].item(), labels[0][0].item())
            # tb_x = epoch_index * len(training_loader) + i + 1
            # tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [59]:
train_set_drug  = dataset(X_train_drug, y_train_drug)
val_set_drug  = dataset(X_val_drug, y_val_drug)
train_dl_drug = DataLoader(train_set_drug, batch_size=1, shuffle=True)
val_dl_drug = DataLoader(val_set_drug, batch_size=1, shuffle=True)
xi, yi = next(iter(train_dl_drug))

ae = AE([xi.shape[1],512,256,512])

print(summary(ae.to("cuda"), xi.shape))
optimizer = torch.optim.Adam(ae.parameters(), lr=1e-4, weight_decay=1e-3)
loss_fn = torch.nn.MSELoss()
# Initializing in a separate cell so we can easily add more epochs to the same run
# timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 100

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    ae.train(True)
    ae = ae.to(device=device)

    avg_loss = train_one_epoch(epoch_number, "writer", train_dl_drug, optimizer, loss_fn)

    running_vloss = 0.0
    # Set the ae to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    ae.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(val_dl_drug):
            vinputs, vlabels = vdata
            vinputs = vinputs.to(device)
            vlabels = vlabels.to(device)
            voutputs = ae(vinputs)
            vloss = loss_fn(voutputs, vlabels) / vinputs.shape[0]
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1) 
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    epoch_number += 1

# with torch.no_grad():
#     for i, data in enumerate(val_dl_drug):
#         inputs, labels = data
#         inputs = inputs.to(device=device)
#         labels = labels.to(device=device)
#         outputs = ae(inputs)
#         print(outputs.detach().to("cpu").numpy(), labels.detach().to("cpu").numpy())

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 512]       1,237,504
              ReLU-2               [-1, 1, 512]               0
           Dropout-3               [-1, 1, 512]               0
            Linear-4               [-1, 1, 256]         131,328
            Linear-5               [-1, 1, 512]         131,584
              ReLU-6               [-1, 1, 512]               0
           Dropout-7               [-1, 1, 512]               0
            Linear-8              [-1, 1, 2416]       1,239,408
Total params: 2,739,824
Trainable params: 2,739,824
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.04
Params size (MB): 10.45
Estimated Total Size (MB): 10.50
----------------------------------------------------------------
None
EPOCH 1:
LOSS train 0.775008580